In [2]:
from collections import namedtuple
import pickle
import re
import numpy as np
import pandas as pd
import datetime

In [3]:
ExtendedUser = namedtuple('extended_user', ['user', 'tweets'])

In [4]:
pickle_file = '../Data/extended_users_attempt3.p'
with open(pickle_file, 'rb') as f:
    extended_users = [ExtendedUser(user, tweets) for user,tweets in pickle.load(f)]

In [5]:
actor_types = pd.read_csv('../Data/Categorized actors.csv')
def collapse_categories(category):
    if category in ['meps', 'europarties']:
        return 'meps & europarties'
    elif category in ['politicians', 'prime_ministers', 'climate_ministers', 'eu_commissioners']:
        return 'assorted politicians'
    elif category in ['eu departments', 'governmental institutions']:
        return 'public institutions'
    else:
        return category
actor_types['collapsed_category'] = actor_types.category.apply(collapse_categories)

In [21]:
status_categories = [
    'user.id',
    'user.name',
    'user.screen_name',
    'user.location',
    'id', 
    'created_at', 
    'favorite_count',
    'retweet_count',
    'entities["user_mentions"]',
    'is_quote_status',
    'lang'
    ]

dict_tweets = {c:[] for c in status_categories}
dict_tweets['text'] = []
dict_tweets['retweeted'] = []
for eu in extended_users:
    for tweet in eu.tweets:
        for c in status_categories:
            dict_tweets[c].append(eval('tweet.'+c))
        try:
            text = tweet.retweeted_status.full_text
            retweeted = tweet.retweeted_status.user.screen_name
        except AttributeError: # If tweet is not a retweet
            text = tweet.full_text
            retweeted = np.nan
        dict_tweets['text'].append(text)
        dict_tweets['retweeted'].append(retweeted)

df_tweets = pd.DataFrame(dict_tweets)
df_tweets['mentions'] = [[u['screen_name'] for u in us] for us in df_tweets['entities["user_mentions"]']]
df_tweets['mentions'] = df_tweets.apply(
    lambda x: [mention for mention in x.mentions if mention!=x.retweeted], axis=1
)
df_tweets = df_tweets.merge(actor_types, how = 'inner', left_on='user.screen_name', right_on='usernames')
df_tweets = df_tweets[df_tweets.lang == 'en']
df_tweets.drop(['usernames', 'entities["user_mentions"]', 'lang'], axis=1, inplace=True)

pickle.dump(
    df_tweets,
    open("../Data/df_tweets_selected_users.p", "wb") 
)
pickle.dump(
    df_tweets[df_tweets.created_at >= '2021-10-01'],
    open("../Data/df_tweets_selected_users_after_oct.p", "wb")
)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=83a05b10-68f2-47cd-90b2-acf816447dd2' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>